In [2]:
!pip install tensorflow==1.15.0 tensorflowjs==1.4.0

  Using cached prompt_toolkit-1.0.14-py3-none-any.whl (248 kB)
ERROR: jupyter-console 6.1.0 has requirement prompt-toolkit!=3.0.0,!=3.0.1,<3.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.14 which is incompatible.
ERROR: ipython 7.16.1 has requirement prompt-toolkit!=3.0.0,!=3.0.1,<3.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.14 which is incompatible.
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 3.0.5
    Uninstalling prompt-toolkit-3.0.5:
      Successfully uninstalled prompt-toolkit-3.0.5


In [3]:
# Download the speech-command model.
!mkdir -p /tmp/tfjs-sc-model
!curl -fsSL https://storage.googleapis.com/tfjs-models/tfjs/speech-commands/v0.3/browser_fft/18w/model.json > /tmp/tfjs-sc-model/model.json
!curl -fsSL https://storage.googleapis.com/tfjs-models/tfjs/speech-commands/v0.3/browser_fft/18w/group1-shard1of2 > /tmp/tfjs-sc-model/group1-shard1of2
!curl -fsSL https://storage.googleapis.com/tfjs-models/tfjs/speech-commands/v0.3/browser_fft/18w/group1-shard2of2 > /tmp/tfjs-sc-model/group1-shard2of2

In [7]:
import tensorflow as tf
import tensorflowjs as tfjs

In [2]:
# Specify the path to the TensorFlow.js Speech Commands model
# (either original or transfer-learned on
# https://teachablemachine.withgoogle.com/)
tfjs_model_json_path = '/tmp/tfjs-sc-model/model.json'

# This is the main classifier model.
model = tfjs.converters.load_keras_model(tfjs_model_json_path)

In [5]:
# Download the audio preprocessing layer that replicates WebAudio's
# Fourier transform for non-browser environments such as Android phones.
!curl -fsSL https://storage.googleapis.com/tfjs-models/tfjs/speech-commands/sc_preproc_model.tar.gz > /tmp/tfjs-sc-model/sc_preproc_model.tar.gz
!cd /tmp/tfjs-sc-model && tar xzvf ./sc_preproc_model.tar.gz

./sc_preproc_model/
./sc_preproc_model/assets/
./sc_preproc_model/variables/
./sc_preproc_model/variables/variables.data-00000-of-00001
./sc_preproc_model/variables/variables.index
./sc_preproc_model/saved_model.pb


In [9]:
# Load the preprocessing layer (wrapped in a tf.keras Model).
preproc_model_path = '/tmp/tfjs-sc-model/sc_preproc_model'
preproc_model = tf.keras.models.load_model(preproc_model_path)
preproc_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
audio_preprocessing_layer (A multiple                  2048      
Total params: 2,048
Trainable params: 0
Non-trainable params: 2,048
_________________________________________________________________


In [12]:
# Construct the new non-browser model by combining the preprocessing
# layer with the main classifier model.
combined_model = tf.keras.Sequential()
combined_model.add(preproc_model)
combined_model.add(model)
combined_model.build([None, 44032])
combined_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      multiple                  2048      
_________________________________________________________________
sequential (Sequential)      (None, 20)                1468684   
Total params: 1,470,732
Trainable params: 1,468,684
Non-trainable params: 2,048
_________________________________________________________________


In [14]:
# Try running some examples through the combined model.
x = tf.random.normal([1, 44032]) / 5.0
combined_model.predict(x)

array([[9.17130828e-01, 1.72089171e-02, 2.65423170e-11, 1.01967939e-11,
        1.62575999e-03, 3.55700441e-02, 9.23496279e-11, 3.30400127e-07,
        2.53620724e-09, 2.09861374e-14, 9.20665741e-07, 2.15095112e-11,
        2.84370966e-02, 1.54009012e-05, 1.17904115e-07, 6.21970520e-09,
        5.10780383e-13, 9.94480165e-10, 1.05076724e-05, 3.98750893e-10]],
      dtype=float32)

In [ ]:
# Save the model as a tflite file.
tflite_output_path = '/tmp/tfjs-sc-model/combined_model.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model(combined_model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS
]
with open(args.output_path, "wb") as f:
  f.write(converter.convert())
print("Saved tflite output at: %s" % args.output_path)